<style>div.container { width: 100% }</style>
<img style="float:left;  vertical-align:text-bottom;" height="65" width="172" src="../../assets/holoviz-logo-unstacked.svg" />
<div style="float:right; vertical-align:text-bottom;"><h2>Exercise 1: Plotting</h2></div>

In this exercise we will try making some of our own visualizations using hvplot.

#### Loading the data 

We will be building a new visualization based on the same data we have cleaned and filtered in the rest of the tutorial. First we load the earthquakes `DataFrame` and filter to those with `>=7` magnitude:

In [ ]:
!python -m pip install "xarray[complete]"

In [10]:
import pathlib
import pandas as pd 
import xarray as xr
import numpy as np

import hvplot.pandas # noqa: adds hvplot method to pandas objects
import hvplot.xarray # noqa: adds hvplot method to xarray objects

df = pd.read_parquet(pathlib.Path('../data/earthquakes.parq'))
df = df.set_index(df.time)
df = df.drop(columns='<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">')

most_severe = df[df.mag >= 7]

In [6]:
most_severe.head()

,depth,depthError,dmin,gap,horizontalError,id,latitude,locationSource,longitude,mag,...,magSource,magType,net,nst,place,rms,status,time,type,updated
time,,,,,,,,,,,,,,,,,,,,,
2000-01-08T16:47:20.580Z,183.4,NaN,NaN,NaN,NaN,usp0009kx3,-16.925,us,-174.248,7.2,...,hrv,mwc,us,NaN,"117 km SSW of Hihifo, Tonga",1.25,reviewed,2000-01-08T16:47:20.580Z,earthquake,2022-04-29T19:41:48.270Z
2000-02-25T01:43:58.640Z,33.0,NaN,NaN,NaN,NaN,usp0009nxg,-19.528,us,173.818,7.1,...,hrv,mwc,us,NaN,None,1.20,reviewed,2000-02-25T01:43:58.640Z,earthquake,2022-04-29T18:33:25.979Z
2000-03-28T11:00:22.510Z,126.5,NaN,NaN,NaN,NaN,usp0009qb4,22.338,us,143.730,7.6,...,hrv,mwc,us,NaN,"Volcano Islands, Japan region",1.22,reviewed,2000-03-28T11:00:22.510Z,earthquake,2022-04-29T18:35:42.761Z
2000-04-23T09:27:23.320Z,608.5,NaN,NaN,NaN,NaN,usp0009rrc,-28.307,us,-62.990,7.0,...,us,mwb,us,NaN,"22 km NW of Añatuya, Argentina",0.89,reviewed,2000-04-23T09:27:23.320Z,earthquake,2022-04-29T19:37:41.054Z
2000-05-12T18:43:18.120Z,225.0,4.6,NaN,NaN,NaN,usp0009suu,-23.548,us,-66.452,7.2,...,hrv,mwc,us,NaN,"75 km N of San Antonio de los Cobres, Argentina",0.86,reviewed,2000-05-12T18:43:18.120Z,earthquake,2022-04-29T19:16:27.106Z


And next we load the population density raster data:

In [7]:
ds = xr.open_dataarray(pathlib.Path('../data/gpw_v4_population_density_rev11_2010_2pt5_min.nc'))
cleaned_ds = ds.where(ds.values != ds.nodatavals).sel(band=1)
cleaned_ds.name = 'population'


In [8]:
ds.head()

<xarray.DataArray (band: 1, y: 5, x: 5)>
array([[[-3.402823e+38, -3.402823e+38, -3.402823e+38, -3.402823e+38,
         -3.402823e+38],
        [-3.402823e+38, -3.402823e+38, -3.402823e+38, -3.402823e+38,
         -3.402823e+38],
        [-3.402823e+38, -3.402823e+38, -3.402823e+38, -3.402823e+38,
         -3.402823e+38],
        [-3.402823e+38, -3.402823e+38, -3.402823e+38, -3.402823e+38,
         -3.402823e+38],
        [-3.402823e+38, -3.402823e+38, -3.402823e+38, -3.402823e+38,
         -3.402823e+38]]], dtype=float32)
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 89.98 89.94 89.9 89.85 89.81
  * x        (x) float64 -180.0 -179.9 -179.9 -179.9 -179.8
Attributes:
    transform:   [ 4.16666667e-02  0.00000000e+00 -1.80000000e+02  0.00000000...
    crs:         EPSG:4326
    res:         [0.04166667 0.04166667]
    is_tiled:    0
    nodatavals:  -3.4028230607370965e+38

#### Visualizing the raster data

Start by using `hvplot.image` to visualize the whole of the population density data using the Datashader support in hvPlot. Grab a handle on this `Image` HoloViews object called `pop_density` and customize it in the `.hvplot.image` call, enabling a logarithmic color scale and a blue colormap (specifically the `'Blues'` colormap). At the end of the cell, display this object.

<br><details><summary>Hint</summary><br>

Don't forget to include `rasterize=True` in the `hvplot.image` call.  You can also use `logz=True`, `clim=(1, np.nan)` and `cmap='Blues'`.

</details>

In [15]:
pop_density = cleaned_ds.hvplot.image(x='x', y='y', rasterize=True, logz=True, clim=(1, np.nan), cmap='Blues')
pop_density  # Display it
cleaned_ds.

:DynamicMap   []
   :Image   [x,y]   (population)

<details><summary>Solution</summary><br>

```python
pop_density = cleaned_ds.hvplot.image(rasterize=True, logz=True, clim=(1, np.nan), cmap='Blues') 
pop_density
```

<br></details>

In [50]:
pop_density = cleaned_ds.hvplot.image(rasterize=True, logz=True, clim=(1, np.nan), cmap='Blues') 
pop_density

:DynamicMap   []
   :Image   [x,y]   (population)

#### Visualizing the earthquake positions

Now visualize the tabular data in `most_severe` by building a `hv.Points` object with `.hvplot`. This will be very similar to the approach shown in the tutorial but this time we want all the earthquakes to be marked with red crosses of size 100. As above, get a handle on the resulting object and display it where the handle is now called `quake_points`.

_Hint: To figure out which arguments you need to pass, try `help(most_severe.hvplot.points)`, then if you can find the right option for marking the points but don't know what value to pass, try passing an invalid value to see from the error message what the valid options are._

<br><details><summary>Hint</summary><br>

Don't forget to map the longitude and latitude dimensions to `x` and `y` in the call to `hvplot.points`.

</details>

In [35]:
import hvplot.pandas # noqa: adds hvplot method to pandas objects
quake_points = most_severe.hvplot.points(x='longitude', y='latitude', marker='+', size=50, color='red')  
quake_points

:Points   [longitude,latitude]

<details><summary>Solution</summary><br>

```python
quake_points = most_severe.hvplot.points(x='longitude', y='latitude', marker='+', size=100, color='red')
quake_points 
```

<br></details>

In [55]:
quake_points.opts(tools=['tap'])

:Points   [longitude,latitude]

#### Composing these visualizations together

Now overlay `quake_points` over `pop_density` to check everything looks correct. Make sure the box select tool is working as expected.

In [53]:
pop_density * quake_points

:DynamicMap   []
   :Overlay
      .Image.I  :Image   [x,y]   (population)
      .Points.I :Points   [longitude,latitude]

<details><summary>Solution</summary><br>
    
```python
pop_density * quake_points
```

<br></details>